In [1]:
import gradio as gr
import re
import subprocess
import os
import glob
import json
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.document_loaders import JSONLoader
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from _config_ import _config_

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
# os.environ["OPENAI_API_KEY"] = "sk-8F9n3GqEgKlV45Js7fE8Bf3285Bc47A6961035F272F3D256"
# os.environ["OPENAI_API_BASE"] = "https://api.aiwaves.cn/v1"

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


loader = JSONLoader(
    # file_path=latest_file,
    file_path='/data/home/Zhifan/gaojiedacheng/data/xhs/17_search_contents_2024-06-02.json',
    jq_schema='.[].desc',
    # content_key='uri'
    text_content=False)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [2]:
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())

OSError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /data/home/Zhifan/anaconda3/envs/travelplan/lib/python3.12/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllmodel.so)

In [ ]:
question = "请生成一个关于重庆2天2000元预算旅游的简单描述脚本"
docs = vectorstore.similarity_search(question)
len(docs)

In [ ]:


retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 定义要查询的问题
# query = f"请生成一个关于{destination}{days}天{budget}元预算旅游的{detail_level}描述脚本"
query = "请生成一个关于重庆2天2000元预算旅游的简单描述脚本"

rag_chain.invoke(query)